In [66]:
# Useful starting lines
%matplotlib inline
import datetime
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [67]:
from proj1_helpers import *
from implementations import*

In [68]:
DATA_TRAIN_PATH = '../data/train.csv/train.csv' # TODO: download train data and supply path here 
y, x, ids = load_csv_data(DATA_TRAIN_PATH)

# Pre-processing


In [69]:
selector = np.all(x != -999.0, axis=1)
x_clean = x[selector]
y_clean = y[selector]

print(x.shape)
print(x_clean.shape)
    

(250000, 30)
(68114, 30)


In [43]:
def standardize_NAN(tX):
    tX_nan = tX.copy()
    for i in range(tX.shape[0]):
        for j in range(tX.shape[1]):
            if (tX_nan[i,j] == -999.0):
                tX_nan[i,j] = np.nan
    return (standardize(tX_nan))


# Tout les nans (correspondant a des valeurs non connues) sont remplacés par la moyenne de la colonnes
def replace_mean(tX_nan):
    means_cols = np.nanmean(tX_nan,axis=1)
    for row in range(0,tX_nan.shape[0]):
        for col in range(0,tX_nan.shape[1]):
            if np.isnan(tX_nan[row,col]):
                tX_nan[row,col]=means_cols[col]
    return (tX_nan)

def get_ind_percentiles(tX, tX_clean, i, percentile):
    arguments = []
    a = np.percentile(tX_clean[:,i],percentile)
    tX_perc = tX.copy()
    arguments = np.argwhere(tX_perc[tX[:,i] > round(a, 2)])
    return list(set(arguments[:,0]))

def remove_rows_by_percentiles(tX,tX_clean):
    args = []
    for i in range(tX.shape[1]):
        args= args+get_ind_percentiles(tX,tX_clean,i,99.97)
    flat_list = [item for item in args]
    mylist = list(set(flat_list))
    return mylist

In [44]:
x_nan, mean_x_nan, std_x_nan = standardize_NAN(x)
x_nan = replace_mean(x_nan)

92.56765498468643


# Subgroup Data

In [36]:
#Feature names
string_features = 'DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,DER_sum_pt,DER_pt_ratio_lep_tau,DER_met_phi_centrality,DER_lep_eta_centrality,PRI_tau_pt,PRI_tau_eta,PRI_tau_phi,PRI_lep_pt,PRI_lep_eta,PRI_lep_phi,PRI_met,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt'
features = string_features.split(",")
dict = {}
for ind, feat in enumerate(features):
    dict[feat] = ind
    
#Subgrouping
x_0=x[x[:,dict['PRI_jet_num']]==0]
x_1=x[x[:,dict['PRI_jet_num']]==1]
x_2=x[x[:,dict['PRI_jet_num']]==2]
x_3=x[x[:,dict['PRI_jet_num']]==3]
y_0=y[x[:,dict['PRI_jet_num']]==0]
y_1=y[x[:,dict['PRI_jet_num']]==1]
y_2=y[x[:,dict['PRI_jet_num']]==2]
y_3=y[x[:,dict['PRI_jet_num']]==3]
ids_0=ids[x[:,dict['PRI_jet_num']]==0]
ids_1=ids[x[:,dict['PRI_jet_num']]==1]
ids_2=ids[x[:,dict['PRI_jet_num']]==2]
ids_3=ids[x[:,dict['PRI_jet_num']]==3]
x_list = [x_0]
x_list.append(x_1)
x_list.append(x_2)
x_list.append(x_3)
ids_list = [ids_0]
ids_list.append(ids_1)
ids_list.append(ids_2)
ids_list.append(ids_3)

#Standardization of subgroups
mean = []
std = []
x_nan_replaced = []
for i in range(4):
    x,m,s = standardize_NAN(x_list[i])
    x_nan_replaced.append(replace_mean(x))
    mean.append(m)
    std.append(s)
    
#Grouping them back again
def group(ls,ids):
    data_ord = np.insert(ls[0],0,ids[0], axis=1)
    for i in range(1,4):
        a = np.insert(ls[i],0,ids[i], axis=1)
        data_ord = np.concatenate((data_ord, a))
    return data_ord[data_ord[:,0].argsort()]
x_inorder = group(x_nan_replaced, ids_list)

44.83469273209879
63.38124719688545
129.81575662840595
139.61904053684216


In [7]:
arg = remove_rows_by_percentiles(x,x_clean)
x_perc = np.delete(x, arg, axis=0)
y_perc = np.delete(y, arg, axis=0)

selected_columns0 = [1, 2, 3, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 29]
selected_columns1 = [1, 2, 3, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 29]
selected_columns_ideal = [0, 1, 2, 3, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 29]

def selected_non_nan_columns(x):
    x_selected = np.zeros((len(x), len(selected_columns0)))
    for i in range(len(x)):
        s = np.take(x[i], indices=selected_columns0, axis=0)
        x_selected[i] = s
    return x_selected

In [56]:
x_sel = selected_non_nan_columns(x)
x_sel.shape

IndexError: index 19 is out of bounds for size 19

In [54]:
v = x_sel != -999
v.all()

True

In [55]:
x_s = x_sel
y_s = y_perc

print(x_s.shape)
print(y_s.shape)

(22164, 19)
(250000,)


# Ridge Regression

In [12]:
x_poly = build_poly(x_s, 2)
lambda_ = 0.1
w, loss = ridge_regression_s(y_s, x_poly, lambda_)
x_poly[:1]

ValueError: shapes (39,22164) and (250000,) not aligned: 22164 (dim 1) != 250000 (dim 0)

In [ ]:
#NOT RUN
indices = build_k_indices(y_s, 4, seed=1)
x_test = x_s[indices[0]]
x_train = np.delete(x_s, [indices[0]], axis=0)
x_train.shape
loss_tr, loss_te = cross_validation_rr(y_s, x_s, k_indices=indices, k=1, lambda_=0.1, degree=3)

In [14]:
K=4
def cross_validation_demo_RR(x,y,K):
    seed = 1
    degree = 4
    k_fold = K
    lambdas = np.logspace(-4, 0, 20)
    
    # split data in k fold
    k_indices = build_k_indices(y, k_fold, seed)
    x_test = x[k_indices[0]]
    x_train = np.delete(x, [k_indices[0]], axis=0)
    
    # define lists to store the loss of training data and test data
    rmse_tr = []
    rmse_te = []
    
    for i in range(len(lambdas)):
        l = lambdas[i]
        avg_err_tr = 0
        avg_err_te = 0
        for k in range(k_fold):
            err = cross_validation_rr(y, x, k_indices, k, l, degree)
            avg_err_tr += err[0]
            avg_err_te += err[1]
        rmse_tr.append(np.sqrt(2 * avg_err_tr / k_fold))
        rmse_te.append(np.sqrt(2 * avg_err_te / k_fold))
    cross_validation_visualization(lambdas, rmse_tr, rmse_te)
    
    min_err_index = 0
    for i in range(1, len(rmse_te)):
        if rmse_te[i] < rmse_te[min_err_index]:
            min_err_index = i
            
    print('Best lambda is: {0}'.format(lambdas[min_err_index]))       


In [ ]:
cross_validation_demo_RR(x_inorder,y,K)

In [ ]:
degree_opt = 4
lambda_opt = 0.0006723357536499335
x_poly = build_poly(x_s, degree_opt)
w_rr_opt, loss_tr = ridge_regression_s(y_s, x_poly, lambda_opt)
print("Training set mse: {}".format(loss_tr))

#Training Accuracy
y_predicted = predict_labels(w_rr_opt, x_poly)
accuracy = []
accuracy.append((list(y_clean == y_predicted).count(True))/len(y_clean))
print("accuracy = {val}".format(val=accuracy))

# 0. Least Squares

In [45]:
def return_factors(x):
    # This function takes a number and prints the factors
    a = []
    for i in range(2,10):
        if x % i == 0:
            a.append(i)
    return a

In [46]:
#tX_LS=tX_clean.copy()
#y_LS= y_clean.copy()
#accuracy=0.7239627683002026 mse=0.7401139564470155

#tX_LS=tX_nan.copy()
#y_LS= y.copy()
# accuracy=0.745088 mse=0.6786542286532609

def compute_least_squares(tX, y):
    tX_LS=tX.copy()
    y_LS= y.copy()

    K_values = return_factors(len(tX_LS))
    accuracy = []
    #K-fold crossvalidation
    for K in K_values:
        #Initialization
        list_tX_LS = np.split(tX_LS,K)
        list_y_LS = np.split(y_LS,K)
        weights=[]
        mse_errors = []
        opt_w = []
        for ind, tX_bloc in enumerate(list_tX_LS):
            tX_test = tX_bloc
            y_test = list_y_LS[ind]
            tX_train = np.concatenate(list_tX_LS[:ind] + list_tX_LS[ind+1:])
            y_train = np.concatenate(list_y_LS[:ind] + list_y_LS[ind+1:])
            mse_LS, optimal_weights_LS = least_squares(y_train,tX_train)
            mse_errors.append(compute_mse(y_test, tX_test, optimal_weights_LS))
            weights.append(optimal_weights_LS)

        opt_w = weights[np.argmin(mse_errors)]
        y_model = predict_labels(opt_w, tX_LS)

        #Computing accuracy
        accuracy.append((list(y_model == y_LS).count(True))/len(y_model))
        print("accuracy = {val} mse={mse}".format(mse = mse_LS, val=accuracy[-1]))

    #Plot of accuracies
    print("\nMaximum accuracy = {val}".format(val=np.max(accuracy)))
    #plt.plot(K_values, accuracy, '.-', markersize=15, label = "Accuracy");
    #plt.xlabel("K value")
    #plt.ylabel("Accuracy")
    #plt.title("Accuracies for Least Squares")
    #plt.legend()

In [47]:
compute_least_squares(x_inorder, y)

accuracy = 0.751196 mse=0.676337897959684
accuracy = 0.750892 mse=0.6772515063201519
accuracy = 0.751328 mse=0.677095521350583
accuracy = 0.751436 mse=0.6771358709269435

Maximum accuracy = 0.751436


# 1.Least squares gradient descent

In [18]:
def cross_validation_LS_GD_demo(x_LS,y_LS,K): 
    #Adding constant term
    tX_LS = np.c_[np.ones((y_LS.shape[0], 1)), x_LS]

    max_iters = 100
    gammas = np.logspace(-4,0,20)


    # Initialization
    w_initial = np.zeros(tX_LS.shape[1])

    list_tX_LS = np.split(tX_LS,K)
    list_y_LS = np.split(y_LS,K)

    gen_opt_w=[]
    gen_mse =[]

    #gamma selection
    for ind, gamma in enumerate(gammas):
        weights=[]
        mse_errors = []
        #K-fold crossvalidation
        for ind, tX_bloc in enumerate(list_tX_LS):
            tX_test = tX_bloc
            y_test = list_y_LS[ind]
            tX_train= list_tX_LS[:ind] + list_tX_LS[ind+1:]
            tX_train= np.concatenate(tX_train)
            y_train= list_y_LS[:ind] + list_y_LS[ind+1:]
            y_train=np.concatenate(y_train)
        
            mse, opt_w = least_squares_GD(y_train, tX_train, w_initial, max_iters, gamma)
            mse_errors.append(compute_mse(y_test, tX_test,opt_w))
            weights.append(opt_w)
        gen_mse.append(np.mean(mse_errors))
        gen_opt_w.append(np.mean(weights, axis=0))
    optimal_gamma_LS_GD = gammas[np.nanargmin(gen_mse)]
    optimal_weights_LS_GD = gen_opt_w[np.nanargmin(gen_mse)]
    mse_LS_GD = np.nanmin(gen_mse)
    print(" gamma={l:.3f},mse={mse:.3f}".format(mse = mse_LS_GD, l = optimal_gamma_LS_GD))

    #Training Accuracy
    y_predicted = predict_labels(optimal_weights_LS_GD, tX_LS)
    accuracy = []
    accuracy.append((list(y_LS == y_predicted).count(True))/len(y_LS))
    print(accuracy)
    #return accuracy,optimal_gamma_LS_GD, optimal_wights_LS_GD,mse_LS_GD


# Least square SDG

We can alter the gamma and the batch size

In [19]:
def cross_validation_LS_SGD_demo(x_LS,y_LS,K):
    #Adding constant term
    tX_LS = np.c_[np.ones((y_LS.shape[0], 1)), x_LS]

    max_iters = 50
    max_batch_size = 32
    gammas = np.logspace(-4,0,20)
    batch_sizes = np.array([2,4,6,8])

    # Initialization
    w_initial = np.zeros(tX_LS.shape[1])
    list_tX_LS = np.split(tX_LS,K)
    list_y_LS = np.split(y_LS,K)


    result_mse =[]
    result_opt_w=[]
    result_gamma=[]
    for ind_batch,batch_size in enumerate(batch_sizes):  
        result_mse_gamma = []
        result_opt_w_gamma = []
        for ind_gamma,gamma in enumerate(gammas):
            mse_errors=[]
            weights=[]
            #K-fold crossvalidation
            for ind, tX_bloc in enumerate(list_tX_LS):
                tX_test = tX_bloc
                y_test = list_y_LS[ind]
                tX_train= list_tX_LS[:ind] + list_tX_LS[ind+1:]
                tX_train= np.concatenate(tX_train)
                y_train= list_y_LS[:ind] + list_y_LS[ind+1:]
                y_train=np.concatenate(y_train)
        
                sgd_mse, opt_w = least_squares_SGD(y_train, tX_train, w_initial, batch_size, max_iters, gamma)
                mse_errors.append(compute_mse(y_test, tX_test,opt_w))
                weights.append(opt_w)
    
            result_mse_gamma.append(np.mean(mse_errors))
            result_opt_w_gamma.append(np.mean(weights,axis=0))
        result_mse.append(np.min(result_mse_gamma))
        result_gamma.append(gammas[np.argmin(result_mse_gamma)])
        result_opt_w.append(result_opt_w_gamma[np.argmin(result_mse_gamma)])

    print(" gamma={l:.3f}, batch={b:.2f}, mse={mse:.3f}".format(mse = np.nanmin(result_mse), l =result_gamma[np.nanargmin(result_mse)], b=np.nanargmin(result_mse)))

    optimal_weights_LS_SGD = result_opt_w[np.nanargmin(result_mse)]
    

    #Training Accuracy
    y_predicted = predict_labels(optimal_weights_LS_SGD, tX_LS)
    accuracy = []
    accuracy.append((list(y_LS == y_predicted).count(True))/len(y_LS))
    print(accuracy)



# Logistic regression (not regularized)

In [20]:
def cross_validation_LR_demo(x_LR,y_LR,K):

    #Adding constant term
    tX_LR = np.c_[np.ones((y_LR.shape[0], 1)), x_LR]
    max_iters = 100
    gammas = np.logspace(-4,0,20)

    # Initialization
    w_initial = np.zeros(tX_LR.shape[1])

    list_tX_LR = np.split(tX_LR,K)
    list_y_LR = np.split(y_LR,K)

    gen_opt_w=[]
    gen_loss =[]

    #gamma selection
    for ind, gamma in enumerate(gammas):
        weights=[]
        loss_errors = []
        #K-fold crossvalidation
        for ind, tX_bloc in enumerate(list_tX_LR):
            tX_test = tX_bloc
            y_test = list_y_LR[ind]
            tX_train= list_tX_LR[:ind] + list_tX_LR[ind+1:]
            tX_train= np.concatenate(tX_train)
            y_train= list_y_LR[:ind] + list_y_LR[ind+1:]
            y_train=np.concatenate(y_train)
            loss, opt_w = logistic_regression(y_train,tX_train,w_initial, max_iters, gamma)
            loss_errors.append(calculate_loss_logistic_reg(y_test, tX_test,opt_w))
            weights.append(opt_w)
        gen_loss.append(np.mean(loss_errors))
        gen_opt_w.append(np.mean(weights, axis=0))


    optimal_gamma_LR = gammas[np.nanargmin(gen_loss)]
    optimal_weights_LR = gen_opt_w[np.nanargmin(gen_loss)]
    print(" gamma={l:.3f},loss={loss:.3f}".format(loss = np.min(gen_loss), l = optimal_gamma_LR))


     #Training Accuracy
    y_predicted = predict_labels(optimal_weights_LR, tX_LR)
    accuracy = []
    accuracy.append((list(y_LR == y_predicted).count(True))/len(y_LR))
    print(accuracy)


# Logistic regression regularized with lambda

In [50]:
def cross_validation_LRR_demo(x_LRR,y_LRR,K):
    
    #Adding constant term
    tX_LRR = np.c_[np.ones((y_LRR.shape[0], 1)), x_LRR]

    max_iters = 50
    lambdas = np.logspace(-4,0,10)
    gammas = np.logspace(-4,0,20)

    # Initialization
    w_initial = np.zeros(tX_LRR.shape[1])
    list_tX_LRR = np.split(tX_LRR,K)
    list_y_LRR = np.split(y_LRR,K)


    result_loss =[]
    result_opt_w=[]
    result_gamma=[]
    for ind,lambda_ in enumerate(lambdas):  
        result_loss_gamma = []
        result_opt_w_gamma = []
        for ind_gamma,gamma in enumerate(gammas):
            loss_errors=[]
            weights=[]
            #K-fold crossvalidation
            for ind, tX_bloc in enumerate(list_tX_LRR):
                tX_test = tX_bloc
                y_test = list_y_LRR[ind]
                tX_train= list_tX_LRR[:ind] + list_tX_LRR[ind+1:]
                tX_train= np.concatenate(tX_train)
                y_train= list_y_LRR[:ind] + list_y_LRR[ind+1:]
                y_train=np.concatenate(y_train)
        
                loss, opt_w = reg_logistic_regression(y_train,tX_train,lambda_,w_initial,max_iters,gamma)
                loss_errors.append(calculate_loss_logistic_reg(y_test, tX_test,opt_w))
                weights.append(opt_w)
    
            result_loss_gamma.append(np.mean(loss_errors))
            result_opt_w_gamma.append(np.mean(weights,axis=0))
        result_loss.append(np.min(result_loss_gamma))
        result_gamma.append(np.argmin(result_loss_gamma))
        result_opt_w.append(result_opt_w_gamma[np.argmin(result_loss_gamma)])

    del result_loss_gamma
    del result_opt_w_gamma
    del loss_errors
    del weights
    print(np.min(result_loss))
    print(result_gamma[np.argmin(result_loss)])
    print(np.argmin(result_loss))
    print(" gamma={l:.3f}, batch={b:.0f}, mse={mse:.3f}".format(mse = np.min(result_loss), l =result_gamma[np.argmin(result_loss)], b=np.argmin(result_loss)))

    optimal_weights_LRR = result_opt_w[np.argmin(result_loss)]
    print(optimal_weights_LRR)

    #Training Accuracy
    y_predicted = predict_labels(optimal_weights_LRR, tX_LRR)
    accuracy = []
    accuracy.append((list(y_LRR == y_predicted).count(True))/len(y_LRR))
    print(accuracy)

In [65]:
cross_validation_LR_demo(x,y,5)

C:\Users\gonxh\Documents\EPFL\Master\MA1\ML\ML_Project1\scripts\implementations.py:20: RuntimeWarning: divide by zero encountered in log
  log = np.sum(y.T@np.log(o)+(1-y.T)@np.log(1-o))


 gamma=0.000,loss=nan
[0.46822786668624283]


C:\Users\gonxh\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:83: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


TEST PART

In [60]:
#With x_te
print("dataset3")
x =x_s.copy()
y =y_s.copy()
print("Regularized Logistic Regression")
cross_validation_LRR_demo(x,y,4)
print("Logistic Regression")
cross_validation_LR_demo(x,y,4)
print("Least-Square -SDG")
cross_validation_LS_SGD_demo(x,y,4)
print("Least-Square -GD")
cross_validation_LS_GD_demo(x,y,4)
print("Ridge Regression")
cross_validation_demo_RR(x,y,4)

dataset3
Regularized Logistic Regression
(22164, 19)
(250000,)


ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [61]:
#With nan value replaced
print("dataset2")
x=x_nan.copy()
y=y.copy()
print("Regularized Logistic Regression")
cross_validation_LRR_demo(x,y,5)
print("Logistic Regression")
cross_validation_LR_demo(x,y,5)
print("Least-Square -SDG")
cross_validation_LS_SGD_demo(x,y,5)
print("Least-Square -GD")
cross_validation_LS_GD_demo(x,y,5)
print("Ridge Regression")
cross_validation_demo_RR(x,y,5)

dataset2
Regularized Logistic Regression
(250000, 30)
(250000,)


MemoryError: 

In [63]:
#With tX and y_LS no corrupted
print("dataset1")
x, m_X,s = standardize(x_clean.copy())
x = x[0:68110]
y = y_clean[0:68110].copy()
print("Regularized Logistic Regression")
cross_validation_LRR_demo(x,y,5)
print("Logistic Regression")
cross_validation_LR_demo(x,y,5)
print("Least-Square -SDG")
cross_validation_LS_SGD_demo(x,y,5)
print("Least-Square -GD")
cross_validation_LS_GD_demo(x,y,5)
print("Ridge Regression")
cross_validation_demo_RR(x,y,5)

dataset1
134.10663740969926
Regularized Logistic Regression
(68110, 30)
(68110,)


MemoryError: 

In [22]:
#Data subgoup
x = x_inorder
y =y.copy()
print("Regularized Logistic Regression")
cross_validation_LRR_demo(x,y,5)
print("Logistic Regression")
cross_validation_LR_demo(x,y,5)
print("Least-Square -SDG")
cross_validation_LS_SGD_demo(x,y,5)
print("Least-Square -GD")
cross_validation_LS_GD_demo(x,y,5)
print("Ridge Regression")
cross_validation_demo_RR(x,y,5)

Regularized Logistic Regression


MemoryError: 

## Generate predictions and save ouput in csv format for submission:

In [ ]:
DATA_TEST_PATH = '../data/test.csv/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [ ]:
OUTPUT_PATH = './logisticRegression_x_te_s' # TODO: fill in desired name of output file for submission
tX_test = np.c_[np.ones((tX_test.shape[0], 1)), tX_test]
tX_test = selected_non_nan_columns(tX_test)
y_pred = predict_labels(optimal_weights_LR, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)